In [1]:
from sqlalchemy import create_engine
import pandas as pd
import os

In [45]:
# Q10 - list of facilities with total revenue less than 1000

# Get list of total revenue by facility
con = engine.connect()
rs = con.execute('''SELECT name, 
                        SUM(CASE WHEN memid = 0 THEN slots * guestcost 
                        ELSE slots * membercost END) AS revenue
                    FROM Bookings
                    INNER JOIN Facilities
                    USING(facid)
                    GROUP BY facid;''')
q10_results = pd.DataFrame(rs.fetchall())
con.close()

In [50]:
# Assign column headers and filter for results
q10_results.columns = rs.keys()
q10_results = q10_results[q10_results['revenue'] < 1000]
q10_results

,name,revenue
3,Table Tennis,180.0
7,Snooker Table,240.0
8,Pool Table,270.0


In [65]:
# Q11 - report of members and who recommended them

# Get a list of members and who recommended them (by member id)
con = engine.connect()
rs = con.execute('''SELECT t.surname, t.firstname, 
                        CASE WHEN recommendedby = 0 THEN recommendedby
                            ELSE (SELECT t2.firstname ||' ' || t2.surname
                                  FROM Members AS t2
                                  WHERE t.recommendedby = t2.memid) END AS recommendedby
                    FROM Members AS t
                    ORDER BY surname, firstname;''')
q11_results = pd.DataFrame(rs.fetchall())
con.close()

q11_results.columns = rs.keys()

q11_results

,surname,firstname,recommendedby
0,Bader,Florence,Ponder Stibbons
1,Baker,Anne,Ponder Stibbons
2,Baker,Timothy,Jemima Farrell
3,Boothe,Tim,Tim Rownam
4,Butters,Gerald,Darren Smith
5,Coplin,Joan,Timothy Baker
6,Crumpet,Erica,Tracy Smith
7,Dare,Nancy,Janice Joplette
8,Farrell,David,None
9,Farrell,Jemima,None


In [67]:
# Q12 - facilities with their usage by member, ignoring guest usage
con = engine.connect()
rs = con.execute('''SELECT name, 
	firstname || ' ' || surname as member,
	COUNT(memid) as num_bookings
FROM Bookings
INNER JOIN Facilities
USING(facid)
INNER JOIN Members
USING(memid)
WHERE memid <> 0
GROUP BY memid
ORDER BY name, member;''')

q12_results = pd.DataFrame(rs.fetchall())
con.close()

q12_results.columns = rs.keys()

q12_results

,name,member,num_bookings
0,Badminton Court,Anna Mackenzie,126
1,Badminton Court,Douglas Jones,14
2,Badminton Court,Erica Crumpet,7
3,Badminton Court,Florence Bader,120
4,Badminton Court,Henry Worthington-Smyth,41
5,Badminton Court,Millicent Purview,16
6,Badminton Court,Nancy Dare,117
7,Massage Room 1,Jack Smith,89
8,Massage Room 1,Janice Joplette,159
9,Massage Room 1,Tim Rownam,408


In [64]:
# Q13 - list facilities with their usage by month, ignoring guest usage
con = engine.connect()
rs = con.execute('''SELECT name,
	SUM(CASE WHEN strftime('%m', starttime) = '01' AND memid <> 0 THEN slots
        	ELSE 0 END) AS january,
    SUM(CASE WHEN strftime('%m', starttime) = '02' AND memid <> 0 THEN slots
        	ELSE 0 END) AS february,
	SUM(CASE WHEN strftime('%m', starttime) = '03' AND memid <> 0 THEN slots
        	ELSE 0 END) AS march,
	SUM(CASE WHEN strftime('%m', starttime) = '04' AND memid <> 0 THEN slots
        	ELSE 0 END) AS april,
	SUM(CASE WHEN strftime('%m', starttime) = '05' AND memid <> 0 THEN slots
        	ELSE 0 END) AS may,
	SUM(CASE WHEN strftime('%m', starttime) = '06' AND memid <> 0 THEN slots
        	ELSE 0 END) AS june,
	SUM(CASE WHEN strftime('%m', starttime) = '07' AND memid <> 0 THEN slots
        	ELSE 0 END) AS july,
	SUM(CASE WHEN strftime('%m', starttime) = '08' AND memid <> 0 THEN slots
        	ELSE 0 END) AS august,
	SUM(CASE WHEN strftime('%m', starttime) = '09' AND memid <> 0 THEN slots
        	ELSE 0 END) AS september,
	SUM(CASE WHEN strftime('%m', starttime) = '10' AND memid <> 0 THEN slots
        	ELSE 0 END) AS october,
	SUM(CASE WHEN strftime('%m', starttime) = '11' AND memid <> 0 THEN slots
        	ELSE 0 END) AS november,
	SUM(CASE WHEN strftime('%m', starttime) = '12' AND memid <> 0 THEN slots
        	ELSE 0 END) AS december,
	SUM(CASE WHEN memid <> 0 THEN slots
        	ELSE 0 END) AS total_mem_bookings
FROM Bookings
INNER JOIN Facilities
USING(facid)
GROUP BY facid;''')
q13_results = pd.DataFrame(rs.fetchall())
con.close()

q13_results.columns = rs.keys()

q13_results

,name,january,february,march,april,may,june,july,august,september,october,november,december,total_mem_bookings
0,Tennis Court 1,0,0,0,0,0,0,201,339,417,0,0,0,957
1,Tennis Court 2,0,0,0,0,0,0,123,345,414,0,0,0,882
2,Badminton Court,0,0,0,0,0,0,165,414,507,0,0,0,1086
3,Table Tennis,0,0,0,0,0,0,98,296,400,0,0,0,794
4,Massage Room 1,0,0,0,0,0,0,166,316,402,0,0,0,884
5,Massage Room 2,0,0,0,0,0,0,8,18,28,0,0,0,54
6,Squash Court,0,0,0,0,0,0,50,184,184,0,0,0,418
7,Snooker Table,0,0,0,0,0,0,140,316,404,0,0,0,860
8,Pool Table,0,0,0,0,0,0,110,303,443,0,0,0,856
